In [1]:
from utils.config import CONFIG
CONFIG.PSP_USE_MEAN = True
import torch
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
from psp.pSp_pure import pSp
from utils.dataset import MulticlassImageDataset
from pathlib import Path
output_dir = Path("output/paper/vs_drr")
output_dir.mkdir(parents=True, exist_ok=True)
dataset = MulticlassImageDataset(
    ["input/data/covid_ct_pneumonia/train"],
    ["lung", "localizer", "bones", "drr", "soft"],
)
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
)

Loading images from each class: 100%|██████████| 5/5 [00:04<00:00,  1.17it/s]


In [2]:
ckpt0 = torch.load("output/psp_pure_pneumonia_chosen_softer/checkpoint/160000.pt", map_location="cuda:0")
net0 = pSp(ckpt0).to("cuda:0")
net0.latent_avg = net0.latent_avg.to("cuda:0") if net0.latent_avg is not None else None
net0.eval();

In [3]:
for idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    with torch.no_grad():
        img_in = batch["lung"].to("cuda:0")
        img_out0, _  = net0(img_in.to("cuda:0"))
        img_out0 = img_out0.to("cuda:0")
    img = torch.concat([img_in,batch["drr"].to("cuda:0"),img_out0], dim=3)
    save_image(
        img,
        str(output_dir / f"{str(idx).zfill(6)}.png"),
        nrow=1,
        normalize=True,
        value_range=(-1, 1),
    )

100%|██████████| 642/642 [00:55<00:00, 11.56it/s]
